In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
from PIL import Image
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score,log_loss, mean_absolute_error,median_absolute_error, mean_squared_error
%matplotlib inline
from tqdm import tqdm
import pickle
def makefile(what,filename):
    with open(filename,"wb") as f3:
        pickle.dump(what,f3)

def readfile(filename):
    with open(filename,"rb") as f4:
        ans=pickle.load(f4)
    return ans
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.metrics import jaccard_score,log_loss, mean_absolute_error,median_absolute_error, mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import CategoricalNB
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

In [12]:
def process(train, name, K, local_size,s1):
    x_train=train[0:50000]
    dim=s1
    training_data=np.array([np.array(x_train[n:n+dim]) for n in range(len(x_train)-(dim-1))])
    kmeans=KMeans(n_clusters=K,max_iter=30)
    kmeans.fit(training_data)
    result=kmeans.predict(training_data)
    clusters=np.zeros(K)
    for value in result:
        clusters[value]+=1
    average=clusters/len(result)*(local_size-1)
    model_name="cluster_model/kmeans_"+name+"_K"+str(K)+"_D"+str(s1)+".pkl"
    proto_name="LP/LP_LS"+str(local_size)+"_K"+str(K)+"_D"+str(s1)+"_"+str(name)+".pkl"
    makefile(kmeans,model_name)
    makefile(average,proto_name)
    testing_data=np.array([np.array(train[n:n+dim]) for n in range(len(train)-(dim-1))])
    result_test=kmeans.predict(testing_data)
    data_test=np.zeros([len(result_test)-(local_size-1),K])
    first=0
    last=0
    for n in range(len(data_test)):
        if n==0:
            for value in result_test[n:n+local_size]:
                first=value
                data_test[n][value]+=1
        else:
            first=result_test[n]
            last=result_test[n+(local_size-1)]
            data_test[n]=data_test[n-1]
            data_test[n][first]-=1
            data_test[n][last]+=1
            
    error=abs(data_test-average).sum(axis=1)
    errorname="UnseenScore/Error_"+name+"_K"+str(K)+"_"+str(local_size)+"_d"+str(s1)+".pkl"
    makefile(error,errorname)

In [13]:


v1=[1,3,6,7,8,9,11,13,17]
WS=[2,3,4,5,6,7,8,9,10]
cl=[2,3,5,10,15,20,25,50]
LS=[50,100,500,660,1000,2000,5000,10000]

In [ ]:
for i in v1:
    data=np.array(pd.read_csv("c"+str(i)+".csv",header=None)[0])
    for window_size in WS:
        print(window_size)
        for K in cl:
            for local_size in LS:
                process(data,str(i),K,local_size,window_size)